In [1]:
import pandas as pd
import numpy as np
import re
import time

import bs4 as bs4
import json

import glob
import tqdm

pd.set_option("max.columns", 131)

#https://strftime.org/
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


CHEGAMOS A PARTE DA MODELAGEM.

Parte que a gente aprende tanto a fazer e vamos ver como perto das outras partes e pequena e relativamente simples de fazer.

Agora vou ter esse novo dataset, onde juntei todos as linhas que a gente ja tinha com anotações e tbm as linhas que nao tinham anotações, e no fim mario fez anotação pra todas as 1100~1200 linhas.

Anteriormente ele nao havia preenchido todas so para mostrar o processo de active learning.

In [2]:
df = pd.read_csv("labels_curso - to_label_2.csv",index_col=0).dropna(subset=['y'])

In [3]:
df.duplicated().mean()

0.0

In [4]:
df.duplicated(['watch-title']).mean()

0.0

In [5]:
df.shape

(1164, 16)

In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [7]:
df_limpo = pd.DataFrame(index=df.index)
df_limpo['title'] = df['watch-title']

# MAIS UMAS VEZ O MESMO PROCESSO DE LIMPEZA DE DADOS, VIEWS E FEATURES

## Limpeza dos dados

In [8]:
clean_date = df['watch-time-text'].str.extract(r"(\d+) de ([a-z]+)\. de (\d+)")
clean_date[0] = clean_date[0].map(lambda x: "0"+x[0] if len(x) == 1 else x)
#clean_date[1] = clean_date[1].map(lambda x: x[0].upper()+x[1:])

mapa_meses = {"jan": "Jan",
              "fev": "Feb",
              "mar": "Mar", 
              "abr": "Apr", 
              "mai": "May", 
              "jun": "Jun",
              "jul": "Jul",
              "ago": "Aug", 
              "set": "Sep", 
              "out": "Oct", 
              "nov": "Nov",
              "dez": "Dec"}

clean_date[1] = clean_date[1].map(mapa_meses)

clean_date = clean_date.apply(lambda x: " ".join(x), axis=1)
clean_date.head()
df_limpo['date'] = pd.to_datetime(clean_date, format="%d %b %Y")

## Limpeza de Views

In [9]:
views = df['watch-view-count'].str.extract(r"(\d+\.?\d*)", expand=False).str.replace(".", "").fillna(0).astype(int)
df_limpo['views'] = views

## Features

In [10]:
features = pd.DataFrame(index=df_limpo.index)
y = df['y'].copy()

In [11]:
features['tempo_desde_pub'] = (pd.to_datetime("2019-12-03") - df_limpo['date']) / np.timedelta64(1, 'D')
features['views'] = df_limpo['views']
features['views_por_dia'] = features['views'] / features['tempo_desde_pub']
features = features.drop(['tempo_desde_pub'], axis=1)

In [12]:
features.head()

,views,views_por_dia
0,28028,61.464912
394,1161,21.109091
393,141646,809.405714
392,325,21.666667
391,61,7.625000


In [13]:
# 555 para treino e 609 pra validação, ainda é um dataset muito pequeno

mask_train = df_limpo['date'] < "2019-04-01"
mask_val = (df_limpo['date'] >= "2019-04-01")

Xtrain, Xval = features[mask_train], features[mask_val]
ytrain, yval = y[mask_train], y[mask_val]
Xtrain.shape, Xval.shape, ytrain.shape, yval.shape

((555, 2), (609, 2), (555,), (609,))

In [14]:
# 

from sklearn.feature_extraction.text import TfidfVectorizer

title_train = df_limpo[mask_train]['title']
title_val = df_limpo[mask_val]['title']

title_vec = TfidfVectorizer(min_df=2, ngram_range=(1,4))
title_bow_train = title_vec.fit_transform(title_train)
title_bow_val = title_vec.transform(title_val)

In [15]:
title_bow_train.shape

(555, 1333)

In [16]:
from scipy.sparse import hstack, vstack

Xtrain_wtitle = hstack([Xtrain, title_bow_train])
Xval_wtitle = hstack([Xval, title_bow_val])

In [17]:
Xtrain_wtitle.shape, Xval_wtitle.shape

((555, 1335), (609, 1335))

## Random Forest

In [18]:
# tirando a quantidade de árvores, que é o n_estimators, que normalmente devemos tentar colocar o maximo possivel que a memoria do PC aguenta
# outro parametro importante é o min_samples_leaf, que determina quantos exemplos precisamos pra criar um novo nó.
# só vou tentar fazer essa divisão(expandir a árvore) se eu tiver pelo menos um exemplo pra colcar no proximo nó(se for =1).
# limitamos a complexidade do modelo definindo parâmetros e assim evitar que o modelo overfite.

mdl = RandomForestClassifier(n_estimators=1000, random_state=0, min_samples_leaf=1, class_weight="balanced", n_jobs=6)
mdl.fit(Xtrain_wtitle, ytrain)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=6, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [19]:
p = mdl.predict_proba(Xval_wtitle)[:, 1]

In [20]:
from sklearn.metrics import roc_auc_score, average_precision_score

In [21]:
average_precision_score(yval, p)

0.21042440914051194

In [22]:
roc_auc_score(yval, p)

0.6897446482278705

##### Para árvores 1000, devemos colocar o maximo de árvores que a memória do PC aguentar. Outro parâmetro importante é o 'min_samples_leaf', ele determina quantos exemplos precisamos para criar um novo nó. Então quando a árvores está fazendo a divisão dos dados pra tentar criar um novo nó, ele fala: só vou tentar fazer a divisão(expandir essa árvore) se eu tiver pelo menos um exemplo pra colocar no próximo nó(se for =1).  Essa é uma forma de regularização, de limitarmos a complexidade do modelo e não overfite.

ap 0.1887361518270232, auc 0.6766232234475438 - min_samples_leaf = 1

##### Ap e auc cairam, mas queremos ver os dois subindo juntos. Como nossos dados são pequenos, mesmo criando nós com com 1 exemplo, os nós acabam ajudando quando criamos esse ensemble, talvez pelo fato de nao termos exemplos o bastante para ter uma redundancia  que nao seja preditiva. O nosso modelo não ta necessariamente memorizando os dados.

ap 0.1773056089416357, auc 0.66359025771068 - min_samples_leaf=2

#### Agora colocando 100 árvores

ap 0.1834073532858915, auc 0.6716990033614437 - n_estimators=100, min_samples_leaf = 1

ap e auc cairam novamente, e as arvores precisam ser o mais profundo possivel, e colocando o maximo de árvores para que se faça a média.

ap 0.1762251704396226, auc 0.6700772542312909 - n_estimators=100, min_samples_leaf=2 


### O que podemos fazer então para melhorar esse modelo ? Mexer no TF-IDF vectorizer, se reduzrimos pra 1, sera que melhora, ou seja se a palavra aparece uma unica vez ela ja ganha uma coluna. Fomos para 1450 palavras, e ai aumentamos bastante o numero de colunas.

ap caiu em relação ao valor anterior com esses parametros da random forest, limitar numero de colunas está ajudando o modelo.

ap 0.17595568029276232, auc 0.6454708969747007 - min_df=1  (estimators = 1000, min_samples_leaf = 1)

vamos testar agora min_df=3, e ele ja começa a perder informação, então vamos ficar no ponto do min_df=2

ap 0.17508239417116303, auc 0.650247685321696 - min_df=3  


Mas tem outro parâmetros interessante de tunar no tf-idf que é o ngram_range, que é o numero de palavras em sequencia que ele vai pegar pra criar bag of words.

##### Então o que ele ta fazendo hoje ?

Por exemplo, se eu tiver um vídeo chamado --> 'intro to machine learning', ele vai criar uma coluna para into, to, machine e learning.

Quando passo o ngram_range, qual é o menor ngrama que se quer e o maior ? Se eu passo (1,2), entao preciso fazer colunas tanto para palavras individuais, mas também para palavras que aparecem juntas. Ou seja, vamos fazer colunas indicando palavras juntas e  pares de palavras, desde que sejam uma depois da outra.

Mario realta que houve casos de (1,8), ou seja, coluna para sequencia de 8 palavras.

Vamos testar, tanto o numero de palavras maximas em sequencia, quanto o minimo, nem sempre o numero minimo 1 nao e tao bom.

#### Exemplo lúdico de disposição de colunas com base no ngram_range.

intro to machine learning -> intro, to, machine, learning -> ngram_range(1,1)

intro to machine learning -> intro, to, machine, learning, intro to, to machine, machine learning -> ngram_range(1,2) 

intro to machine learning -> intro to, to machine, machine learning -> ngram_range(2,2) *apenas colunas com 2 palavras

*é interessante da uma retunada no min_df depois que definiu o numero de ngramas, pois ele muda, já que agora o contexto das colunas é outro.

ap 0.21140611241061869, auc 0.6785398360559061 - min_df=2, ngram_range=(1,2)  
ap 0.22228951304206077, auc 0.6914990859232175 - min_df=2, ngram_range=(1,3) * melhor situação que conseguimos aqui 
ap 0.20564709615419985, auc 0.6848794008374124 - min_df=2, ngram_range=(1,4)  *aqui ela ja começou a cair


# LightGBM - MODELO FAVORITO DO MARIO

Geralmente bate a random forest, mas tem que tunar mais parâmetros

In [23]:
from lightgbm import LGBMClassifier

In [24]:
mdl = LGBMClassifier(random_state=0, class_weight="balanced", n_jobs=6)
mdl.fit(Xtrain_wtitle, ytrain)

LGBMClassifier(boosting_type='gbdt', class_weight='balanced',
               colsample_bytree=1.0, importance_type='split', learning_rate=0.1,
               max_depth=-1, min_child_samples=20, min_child_weight=0.001,
               min_split_gain=0.0, n_estimators=100, n_jobs=6, num_leaves=31,
               objective=None, random_state=0, reg_alpha=0.0, reg_lambda=0.0,
               silent=True, subsample=1.0, subsample_for_bin=200000,
               subsample_freq=0)

In [25]:
p = mdl.predict_proba(Xval_wtitle)[:, 1]

/home/jarbasjr/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:546: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


In [26]:
average_precision_score(yval, p), roc_auc_score(yval, p)

(0.19802541702844073, 0.6238131745002063)

# Bayesian Optimization

Vamos otimizar o LightGBM.

Mario NÃO GOSTA DE USAR GRIDSEARCH, nem o modelo Nayve Baises(muito fraco segundo mario).

Grid Search demora muito, busca exaustiva.


ALTERNATIVAS: Random Search, busca aleatória, selecionamos aleatoriamente algumas combinçãoes dentro de um intervalo razoavel e testam elas. Papers, artigos e etc dizem que random search e mais eficiente que grid search. Trazem resultado melhor em bem menos tempo que GS.

Existem algumas técnicas que conseguem ser iguais ou melhores que random search, como o BAYESIAN OPTMIZATION.

Nesse caso, o mecanismo que o algortimo de busca vai fazer é: primeiro faz uma busca aleatória, e a partir de um determinado número de amostras que ele tiver de pares de parâmetros e a métrica de erro, ele vai treinar o modelo para que através dos parametros que a gente ta tentando tunar ele tentar prever qual vai ser a métrica de erro.


Em 'n_random_starts = 20', a gente vai pegar 20 combinações de parametros aleatoriamente, e com esses 20 a gente vai criar um modelo que vai ser um enseble de árvores, e ai a gente vai gerar 10 mil combinçãoes e vai mandar esse modelo prever qual a combinação que pode me dar o maior ganho, e ele vai testar essa com a proxima combinação na nossa busca.


É IMPORTANTE, pq tunar modelos leva um tempo razóavel, imagine que o modelo leve 30 min pra rodar, 3 ou 4h, nao da pra testar muitas combinações dependendo do tamanho do dataset. Entao temos que ter uma busca mais direcionada, e combinações que tem maior chance de dar maior ganho.

Ainda não há resultado claro que a bayesian optimazation seja melhor que a random search, mas segundo Mario, em 20, 30 passos acaba dando uma solução boa e ele prefere usar BO.


Os parÂmetros ta TF-IDF tbm podem ser tunados, e ele vai tentar tunar tudo aqui junto, nao só do LightGBM, mas do TF-IDF tbm.

Antigamente era mais dificil fazer esse tipo de coisa, mas conforme essas buscas ficam mais eficientes, acaba valendo a pena tentar achar todos os parÂmetros.


Inclusive, alguns dos melhores resultado com deep learning, são redes neurais com suas arquiteturas encontradas através de buscas mais inteligentes que a aleatória.





In [27]:
from skopt import forest_minimize

In [28]:
# No lightGBM Mario gosta de tunar 5 parâmetros, learning rate, profundidade máxima das árvores , numero minimo de exemplos em cada nó
# subsample, se vc treina todas as arvores com a mesma amostra, elas tendem a ter mais overfiting(memorizar mais os dados), agora se vcs pega uma amostra aleatoria toda vez que for treinar uma nova árvore, ele tende a regulariza um pouco, reduzindo o overfiting, isso é subsample 
# colsample_bytree, mesma coisa que sample, só que para coluna, a cada arvore criada, ele seleciona uma parte das colunas para evitar overfiting
# vamos tunar tbm o numero de árvores: n_estimators

# em RF-IDF vamos tunar min_df e ngram_range(poderia ter colocado um pro minimo tbm, desde que fosse menor que o do maximo).
def tune_lgbm(params):
    print(params)
    lr = params[0]
    max_depth = params[1]
    min_child_samples = params[2]
    subsample = params[3]
    colsample_bytree = params[4]
    n_estimators = params[5]
    
    min_df  = params[6]
    ngram_range = (1, params[7])
    
    title_vec = TfidfVectorizer(min_df=min_df, ngram_range=ngram_range)
    title_bow_train = title_vec.fit_transform(title_train)
    title_bow_val = title_vec.transform(title_val)
    
    Xtrain_wtitle = hstack([Xtrain, title_bow_train])
    Xval_wtitle = hstack([Xval, title_bow_val])
    
    mdl = LGBMClassifier(learning_rate=lr, num_leaves=2 ** max_depth, max_depth=max_depth, 
                         min_child_samples=min_child_samples, subsample=subsample, colsample_bytree=colsample_bytree,
                         bagging_freq=1, n_estimators=n_estimators, random_state=0, class_weight="balanced", n_jobs=6)
    
    mdl.fit(Xtrain_wtitle, ytrain)
    
    p = mdl.predict_proba(Xval_wtitle)[:, 1]
    
    print(roc_auc_score(yval, p))
    
    return -average_precision_score(yval, p)


space = [(1e-3, 1e-1, 'log-uniform'), #lr, 'log-uniform' pq ao invés de querer que ele pegue de uma distribuição uniforme, ou seja, que cada valor entre esse intervalos tenha a mesma chance de fazer parte de uma combinação, eu quero que ele pegue valores mais próximo da banda inferior, quanto menor o valor da lr, tende a estabilizar o modelo melhor  
        # todas as outras seguem uma distribuição uniforme entre a faixa de valores
        # se eu ver que o melhor parametro ta no limite de algum desses intervalos(exatamente uma das bandas), ai tento expandir a busca
        # mario tem experiencia com lightgbm, entao ele sabe que esses sao os valores mais ou menos
        # mas com redes neurais e dificil supor a faixa da arquitetura mais precisa, mesmo para os maiores especialistas
        # ai se coloca numeros maiores no intervalo, e ai vai fazendo uma busca cada vez menor
        (1, 10), # max_depth
        (1, 20), # min_child_samples
        (0.05, 1.), # subsamples
        (0.05, 1.), # colsample_bytree
        (100, 1000), # n_estimators
        (1, 5), # min_df
        (1, 5)] # ngram_range

# vamos rodar isso por 50 iterações e ver o resultado.
# na função forest_minimize, o que devemos passar ?
# passamos uma função que aceite uma combinação/lista de parametros e retorne um número que é a pontuação que queremos minimizar.
# estamos retornando a negativa da average_precision, pq queremos maximizar a métrica, e se tu tem um mecanismo que minimza, então tu coloca um negativo na minimzação, mesma coisa que maximizar
# bom fazer print dos parametros e de outra métrica que estamos utilizando pra ter certeza que ta tudo certo, e ver uma combinação de parametros que melhore ambas métricas

# além da função de minimização, precisamos passar o 'espaço' de intervalos, lista de tuples, nesse caso colocamos comentado o que cada lista do elemento se refere
# os valores que chutamos aprendemos com a experiencia, ou vendo artigos que sugerem
res = forest_minimize(tune_lgbm, space, random_state=160745, n_random_starts=20, n_calls=50, verbose=1)

Iteration No: 1 started. Evaluating function at random point.
[0.009944912110647982, 5, 1, 0.4677107511929402, 0.49263223036174764, 272, 3, 1]


/home/jarbasjr/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:546: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


0.6016099545910243
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 0.5845
Function value obtained: -0.1404
Current minimum: -0.1404
Iteration No: 2 started. Evaluating function at random point.
[0.053887464791860025, 1, 15, 0.7437489153990157, 0.8675167974293533, 549, 3, 4]


/home/jarbasjr/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:546: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


0.5507312614259597
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 0.3205
Function value obtained: -0.1203
Current minimum: -0.1404
Iteration No: 3 started. Evaluating function at random point.
[0.004151454520895999, 6, 20, 0.8682075103820793, 0.9491436163200662, 411, 4, 3]


...

0.6480214660612137
Iteration No: 48 ended. Search finished for the next optimal point.
Time taken: 2.9721
Function value obtained: -0.1582
Current minimum: -0.2378
Iteration No: 49 started. Searching for the next optimal point.
[0.06892466655245144, 7, 5, 0.8261588403741187, 0.07729616979076112, 853, 2, 5]


/home/jarbasjr/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:546: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


0.6603467594503744
Iteration No: 49 ended. Search finished for the next optimal point.
Time taken: 1.9908
Function value obtained: -0.1719
Current minimum: -0.2378
Iteration No: 50 started. Searching for the next optimal point.
[0.06602730762014003, 10, 2, 0.6724229168392452, 0.07445872598097698, 855, 2, 5]


/home/jarbasjr/anaconda3/lib/python3.7/site-packages/lightgbm/basic.py:546: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


0.6782449725776964
Iteration No: 50 ended. Search finished for the next optimal point.
Time taken: 4.5583
Function value obtained: -0.2132
Current minimum: -0.2378


#### NA ITERAÇÃO 44, batemos ap, mas nao batemos auc.

referencia anteior - ap 0.22228951304206077, auc 0.6914990859232175 - min_df=2, ngram_range=(1,3) 

 após tunagem - ap 0.2378, auc 0.6883293035324645 LGBM - [0.08265121231498246, 7, 1, 0.7251351011494334, 0.07547006552546137, 839, 2, 5]


Por isso que é bom imprimir os parâmetros, pode copia-la, ou usamos 'res.x', nos da a lista de parametros com o melhor score

In [29]:
# em ordem: lr, max_depth =7, min_child_samples, subsamble, colsample_bytree, n_estimators, min_df e ngram_range.
res.x

[0.08265121231498246, 7, 1, 0.7251351011494334, 0.07547006552546137, 839, 2, 5]

A gente deve usar esse modelo em produção ??????????

Antes de definir quais modelos vão pra produção, eu vou testar uma Logistic Regression.

Então a gente já sabe que esses parametros do LGBM sao os que funcionam para o melhor score, vamos anotar o da random forest.


##### Random Forest: ap 0.22228951304206077, auc 0.6914990859232175 , parametros - min_df=2, ngram_range=(1,3)


##### LGBM:  ap 0.2378, auc 0.6883293035324645, parametros -  [0.08265121231498246,  7,  1,  0.7251351011494334,  0.07547006552546137,  839,  2,  5]


# REGRESSÃO LOGÍSTICA


Primeiro vamos mostrar como transforma quando se tem colunas esparsas e numericas, e depois ver se a reg log bate os modelos mais complexos que ja testamos. 

In [30]:
# MaxAbsscaler - ele escala os valores dividindo pelo valor maximo absoluto que encontrar em cada coluna. 
# StandardScaler - faz subtração da media e divide pelo desvio padrao, mas ai perdemos a esparsidade, a estrutura cheia de zeros nessa matriz e nao vale a pena
# MaxAbs e recomendado pra isso mas mesmo assim vamos testar ambas normalizações no caso da reg log

from sklearn.preprocessing import MaxAbsScaler, StandardScaler
from scipy.sparse import csr_matrix

In [31]:
# como a gente tem colunas esparsas e densas, transformei o Xtrain_wtitle usa os mesmo parametros que usamos no tf-idf da random forest
# transformei pra sparsa pra poder usar esses scalers

Xtrain_wtitle2 = csr_matrix(Xtrain_wtitle.copy())
Xval_wtitle2 = csr_matrix(Xval_wtitle.copy())

#scaler = StandardScaler()
scaler = MaxAbsScaler()



# a primeira coisa que fizemos é fazer o scaling para as variaveis numericas, e aqui pegamos so as colunas 0 e 1, e eu so to escalando elas
# só to escalando essas colunas pq nao quero usar nas minhas colunas esparsas pq vai tirar os meu zeros e quebrar a minha estrutura
#Xtrain_wtitle2[:, :2] = scaler.fit_transform(Xtrain_wtitle2[:, :2].todense())
#Xval_wtitle2[:, :2] = scaler.transform(Xval_wtitle2[:, :2].todense())


# Aqui é o scaling com MaxAbs, que pode ser para toda matriz, nao afetando as colunas esparsas
Xtrain_wtitle2 = scaler.fit_transform(Xtrain_wtitle2)
Xval_wtitle2 = scaler.transform(Xval_wtitle2)

In [32]:
Xval_wtitle2.shape

(609, 1335)

In [33]:

mdl = LogisticRegression(C=0.5, n_jobs=6, random_state=0)
mdl.fit(Xtrain_wtitle2, ytrain)

LogisticRegression(C=0.5, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=6, penalty='l2', random_state=0,
                   solver='lbfgs', tol=0.0001, verbose=0, warm_start=False)

In [34]:
p = mdl.predict_proba(Xval_wtitle2)[:, 1]

In [35]:
average_precision_score(yval, p), roc_auc_score(yval, p)

(0.2162535946903888, 0.6861178274458927)

primeiro testando sem nenhum tuning, e temos metricas que não estão ruins.
##### (0.20616279322296893, 0.6606416229285841) - sem tuning, standardscaler

agora trocando o normalizador, nao preciso fazer o scaling so nas duas primeiras colunas, mas em toda matriz.
resultado um pouco melhor do que com standard scaler.
##### (0.20757989629841797, 0.6862357728371764) - sem tuning, maxabsscaler

como quero uma solução rapida, so vou tunar o parametro C, que é a penalidade, coef que multiplica pela penalidade de regularização da reg log. Se ela tivesse dado um resultado muito melhor, talvez nos tunassemos com mais cuidado, mas aqui ja temos dois modelos que são legais. Por padrão é C=1, é uma medida inversa da regularização, quanto maior ele for menor a regularização. Primeior vamos reduzir a regularização e colocar C=10. Resultado muito pior, entao precisamos da regularização proxima de 1 ou até menor que 1.
##### (0.18953863996413214, 0.6741463702305833) - C=10, maxabsscaler


Aqui no caso, C menor que 1, ja que performou mal com regularização reduzida, mais regularização está nos ajudando
##### (0.21340786207179874, 0.6835525151854692) - C=0.5, maxabsscaler

A gente poderia continuar tunando pra achar valores melhores e etc. C=0.4, C=0.6.

Tem outras coisas que poderiamos tunar mas aqui estamso satisfeitos.


Só que, eu poderia escolher um desses modelos pra colocar em produção ou colocar mais de um. Aqui eu nao tenho previsões que precisam ser extremamente rapidas, entao podemos usar um ENSEMBLE pra fazer a previsão final da relevancia de cada video, o rankeamento de cada video. Entaõ vou pegar alguns ou todos esses modelos e vou fazer uma média deles pra ver se conseguimos uma solução ainda melhor, não usando apenas individualmente Random Forest, LGBM ou RegLog, mas uma média ponderada entre eles.

Outra vantagem é que teremos uma solução mais robusta, se fizermos a média de modelos que estão errando em lugares diferentes, podemos ganhar perfomance em termos de métrica e ganhar mais estabilidade e robustez na nossa solução.